## 공급량 데이터 수집 및 저장 (특정 연도와 월 지정)
+ 작성: 임경호

In [12]:
import pandas as pd

file = '천연가스공급량(일별)_8월까지확정_230905.xlsx'
path = 'D:/PythonProject/data-gatherer/dse/data/' + file
year = 2023
month = 8
sheet_m3 = '2023_부피'
sheet_mj = '2023_열량'

header_row = 2      # Header (세번째 행은 2)
num_of_rows = 31    # Data 행 개수

In [13]:
# 특정 연도와 월의 해당일자(1일 ~ 말일)의 공급량 데이터 추출
def read_data(path, sheet, year, month):
    # 읽을 데이터 열: 첫번째 열('구 분') 및 n번째 열(month)
    use_cols= [0, month]        
    df = pd.read_excel(path, sheet_name=sheet, header=header_row, usecols=use_cols, nrows=num_of_rows)
    # 인덱스를 '구분'셀로 변경
    df = df.set_index('구 분')  
    # 데이터값이 NaN인 경우 삭제 (예: 6월인 경우 31일 데이터 없으므로 삭제)
    df.dropna(subset = [month], inplace=True)
    
    #임의의 데이터프레임 생성 >> 데이터를 전처리하여 넣을 곳
    new_df = pd.DataFrame(columns=['year','month','day','supply']) 
    for j in df.index:  #j는 일(day)
        supply_value = df.loc[j][month]        
        if supply_value > 0:    # 값이 있는 경우 및 값이 0보다 큰 경우만 해당됨
            array=[]            # 임시 배열 추가(값이 계속 초기화됨)
            array.extend([year, month, j, supply_value])    # 데이터프레임에 값을 추가하기 위해 형식 변경
            result1 = "".join(map(str, array))              # 고유값 생성(인덱스 사용을 위해서)
            new_df.loc[result1] = array         # 고유값의 인덱스에 값을 넣어줌
    
    new_df = new_df.reset_index(drop=True) #인덱스 초기화
    new_df['date'] = pd.to_datetime(new_df[['year', 'month', 'day']], format = '%Y%M%D')
    new_df = new_df.drop(labels=['year','month','day'],axis=1) #임시로 생성한 날짜 형식의 데이터 제거
    new_df['date_id'] = new_df['date'].dt.strftime('%Y%m%d') #데이터 형식 변경
    new_df = new_df.drop(labels=['date'],axis=1) #임시로 생성한 날짜 형식의 데이터 제거
    new_df = new_df.reindex(['date_id', 'supply'], axis = 1)
                
    return new_df

In [14]:
df_supply_m3 = read_data(path, sheet_m3, year, month)
df_supply_m3.rename(columns={'supply':'supply_m3'},inplace=True)
df_supply_mj = read_data(path, sheet_mj, year, month)
df_supply_mj.rename(columns={'supply':'supply_mj'},inplace=True)

In [15]:
df_supply_mj.shape, df_supply_m3.shape

((31, 2), (31, 2))

In [16]:
# 데이터 결합
df_supply = pd.merge(df_supply_mj, df_supply_m3, left_on='date_id', right_on='date_id', how='outer')
df_supply

,date_id,supply_mj,supply_m3
0,20230801,56411787,1317286
1,20230802,57997179,1355885
2,20230803,60307975,1409669
3,20230804,59256816,1386284
4,20230805,48469409,1134906
5,20230806,43174093,1012300
6,20230807,62116040,1456886
7,20230808,66209473,1550748
8,20230809,66522625,1559669
9,20230810,65075871,1527359


In [17]:
import sys
module_path = "D:\PythonProject\data-gatherer\common"
sys.path.append(module_path)
import dbconnect

conn = dbconnect.db_connect("DEMO_DW")
cur = conn.cursor()

for row in df_supply.itertuples():
    sql = "insert into supply(date_id, supply_mj, supply_m3) values (%s, %s, %s)"
    cur.execute(sql, (row[1], row[2], row[3]))

conn.commit()
conn.close()